<a href="https://colab.research.google.com/github/Gmarco2706/DNAGYMCraft/blob/main/GymGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

# Caricamento del nuovo dataset
gym_dataset2 = pd.read_csv('gym_dataset2.csv')

gym_dataset2

In [ ]:
print(gym_dataset2.columns)

In [ ]:
#input
livello_di_abilita = input("Inserisci il tuo livello di abilità (Principiante/Intermedio/Esperto): ")
luogo_di_allenamento = input("Ti alleni in palestra o a casa? (Palestra/Casa): ")

fitness_storico = []


<h1>Creazione e inizializzazione della popolazione </h1>

<p>
creiamo una popolazione iniziale di schede di allenamento, ciascuna con una struttura che include giorni settimanali e esercizi casuali selezionati dal dataset. La funzione <b>generate_random_workout</b> genera una scheda di allenamento casuale con 7 esercizi rispettando il vincolo di avere un esercizio di stretching e di avere un max di 7 esercizi al giorno compreso lo stretching. La funzione <b>generate_initial_population</b> crea una popolazione di schede di allenamento settimanali per gli individui. Alla fine stampiamo la prima scheda
</p>

In [ ]:
#creazione della popolazione
def generate_random_workout(data, max_exercises=7, include_stretching=True):

    workout = []
    #estrazione gli esercizi di stretching dal dataset
    stretching_exercises = data[data['Type_Stretching'] == 1]

    #aggiunge un esercizio di stretching random (vincolo)
    if include_stretching and not stretching_exercises.empty:
        workout.append(stretching_exercises.sample(n=1).iloc[0])

    remaining_slots = max_exercises - len(workout)
    #estrazione degli esercizi che non sono di stretching
    other_exercises = data[data['Type_Stretching'] == 0]
    #filla tutti gli slot vuoti rimanenti (vincolo max 7 es)
    workout += other_exercises.sample(n=min(len(other_exercises), remaining_slots)).to_dict('records')

    return workout
#genera la prima generazione di schede di allenamento
def generate_initial_population(data, population_size=10, days_per_week=3):
    population = []
    #per ogni individuo crea schede di allenamento
    for _ in range(population_size):
        weekly_schedule = {}
        for day in range(days_per_week):
            daily_workout = generate_random_workout(data)
            weekly_schedule[f"Day {day + 1}"] = daily_workout
        population.append(weekly_schedule)

    return population

# Generiamo una popolazione iniziale di schede di allenamento
initial_population = generate_initial_population(gym_dataset2)
initial_population[0]  # Visualizziamo la scheda



<h1>Funzione di Fitness</h1>


<p>
creiamo  la funzione di fitness che valuta una scheda di allenamento in base a vari criteri: (1)livello di abilità, (2)varietà di tipi di esercizi, (3)la varietà di gruppi muscolari coinvolti, (4)il luogo di allenamento, (5)feedback fornito.
La funzione calcola un punteggio totale in base a questi criteri e lo restituisce come risultato.
</p>

In [ ]:
#funzione di fitness
def fitness(scheda, livello_di_abilita, luogo_di_allenamento, pesi, feedback = None):
    punteggio = 0
    #creo i vari livelli di abilità associandoli alle colonne del dataset
    livello_mapping = {"Principiante": "Level_Beginner", "Intermedio": "Level_Intermediate", "Esperto": "Level_Expert"}
    livello_colonna = livello_mapping[livello_di_abilita]
    #iniziamo a calcolare il punteggio basandoci sull'aderenza al livello di abolità dell'utente
    for day in scheda:
        for esercizio in scheda[day]:
            if esercizio[livello_colonna] == 1:
                punteggio += pesi['livello_aderenza']
    #calcoliamo il punteggio sta volta sulla base della differenziazione degli esercizi e dei gruppi muscolari
    tipi_esercizi = set()
    gruppi_muscolari = set()
    for day in scheda:
        for esercizio in scheda[day]:
            for tipo in ["Type_Cardio", "Type_Strength", "Type_Stretching"]:
                if esercizio[tipo] == 1:
                    tipi_esercizi.add(tipo)
            for gruppo in ["BodyPart_Abdominals", "BodyPart_Biceps", "BodyPart_Calves", "BodyPart_Chest", "BodyPart_Forearms", "BodyPart_Glutes", "BodyPart_Hamstrings", "BodyPart_Lats", "BodyPart_Lower Back", "BodyPart_Middle Back", "BodyPart_Neck", "BodyPart_Quadriceps", "BodyPart_Shoulders", "BodyPart_Traps", "BodyPart_Triceps"]:
                if esercizio.get(gruppo, 0) == 1:
                    gruppi_muscolari.add(gruppo)

    punteggio += len(tipi_esercizi) * pesi['varietà_tipo'] + len(gruppi_muscolari) * pesi['varietà_gruppo']
    #andiamo a modificare il punteggio in base all'input del luogo di allenamento scegliendo gli esercizi senza attrezzi per chi si allena da casa
    if luogo_di_allenamento == "Casa":
        for day in scheda:
            for esercizio in scheda[day]:
                if esercizio["Equipment_None"] == 1:
                    punteggio += pesi['attrezzatura_casa']


    # Modifica del punteggio in base al feedback
    if feedback:
        if feedback == "Ottima":
            punteggio += 10  # Incoraggia schede simili
        elif feedback == "Troppo intensa":
            punteggio -= 5  # Penalizza schede troppo intense
        elif feedback == "Troppo facile":
            punteggio -= 6 #penalizza schede troppo facili

    return punteggio


<h1> K-Tournament Selection </h1>

<p>
per la selezione degli individui abbiamo pensato di utilizzare il K-Tournament Selection. Creando una serie di tornei dove inseriamo 3 individui, alla fine selezioneremo i migliori individui  per farli diventare i genitori della prossima generazione. La probabilità di scelta è data dal valore di fitness di ogni individuo.
</p>

In [ ]:
#selezione tramite K-Tournament selection
def selezione_tornei(popolazione, valutazioni_fitness, numero_selezionati):
    selezionati = []
    for _ in range(numero_selezionati):
        #selezioniamo K partecipanti casuali dove k= 3 insieme al loro valore di fitness
        partecipanti_torneo = random.sample(list(zip(popolazione, valutazioni_fitness)), k=3)
        #prendiamo il migliore di loro e lo selezioniamo
        migliore = max(partecipanti_torneo, key=lambda individuo: individuo[1])[0]
        selezionati.append(migliore)

    return selezionati


<h1> One-Point Crossover </h1>

<p>
Creiamo un figlio vuoto, successivamente settiamo randomicamente un punto di crossover, creando una scheda di allenamento per il figlio che combini le due parti dei genitori prima e dopo il punto di crossover.
</p>

In [ ]:
#Funzione per eseguire la crossover di due genitori
def crossover(genitore1, genitore2):
    figlio = {}
    punto_crossover = random.randint(1, len(genitore1))

    for i, (day, workout) in enumerate(genitore1.items()):
        if i < punto_crossover:
            #copiamo tutto quello che c'è prima del punto di crossover di entrambi i genitori
            figlio[day] = workout
        else:
            figlio[day] = genitore2[day]

    return figlio

<h1> Mutazione <h1>

<p>
Iteriamo i giorni e per ognuno di essi viene generato un numero casuale e se questo numero è minore del <b>tasso_di_mutazione</b> viene eseguita la mutazione. Quindi viene scambiato casualmente un esercizio nella scheda con un esercizio randomico del dataset
</p>

In [ ]:
#Funzione per eseguire una mutazione su una scheda di allenamento
def mutazione(scheda, gym_dataset2, tasso_di_mutazione=0.1):
    #Creaiamo una copia della scheda originale
    nuova_scheda = scheda.copy()
    #iteriamo i giorni, il tasso di mutazione definisce la probabilità di mutare l'allenamento e viene scelto a caso un esercizio da sostituire
    for day, allenamento in nuova_scheda.items():
        if random.random() < tasso_di_mutazione:
            indice_esercizio = random.randint(0, len(allenamento) - 1)
            nuovo_esercizio = gym_dataset2.sample(n=1).iloc[0].to_dict()
            nuova_scheda[day][indice_esercizio] = nuovo_esercizio
    return nuova_scheda

<h1> Funzione dell'algoritmo genetico <h2>
<p>
viene generata una popolazione di schede, e iniziamo ad iterare su ogni generazione selezionando i genitori migliori per creare la next gen,
in fine restituiamo la migliore scheda trovata
</p>

In [ ]:
def algoritmo_genetico(data, livello_di_abilita, luogo_di_allenamento, feedback=None, numero_generazioni=50, dimensione_popolazione=10, tasso_di_mutazione=0.1):
    popolazione = generate_initial_population(data, population_size=dimensione_popolazione)
    pesi_iniziali = {'livello_aderenza': 1.0, 'varietà_tipo': 1.0, 'varietà_gruppo': 1.0, 'attrezzatura_casa': 1.0}

    fitness_generazione = []

    for generazione in range(numero_generazioni):
        valutazioni_fitness = [fitness(scheda, livello_di_abilita, luogo_di_allenamento, pesi_iniziali, feedback) for scheda in popolazione]


        genitori = selezione_tornei(popolazione, valutazioni_fitness, numero_selezionati=len(popolazione) // 2)
        nuova_generazione = []

        while len(nuova_generazione) < dimensione_popolazione:
            genitore1, genitore2 = random.sample(genitori, 2)
            figlio = crossover(genitore1, genitore2)
            figlio = mutazione(figlio, data, tasso_di_mutazione)
            nuova_generazione.append(figlio)



        popolazione = nuova_generazione
        miglior_scheda = max(popolazione, key=lambda scheda: fitness(scheda, livello_di_abilita, luogo_di_allenamento, pesi_iniziali))
        fitness_miglior_scheda = fitness(miglior_scheda, livello_di_abilita, luogo_di_allenamento, pesi_iniziali)
        fitness_generazione.append(fitness_miglior_scheda)

        print(f"Generazione {generazione + 1}: Fitness Migliore = {fitness_miglior_scheda}")

    print(f"Fitness Migliore alla Fine dell'Algoritmo: {fitness_generazione[-1]}")
    return miglior_scheda


<h1> Stampe delle schede e feedback <h1>

<p>
stampiamo le schede e inseriamo un input di feedback per poi stampare altre schede modificate rispetto al feedback dato
</p>

In [ ]:
# Visualizzazione della migliore scheda di allenamento
pesi_iniziali = {'livello_aderenza': 1.0, 'varietà_tipo': 1.0, 'varietà_gruppo': 1.0, 'attrezzatura_casa': 1.0}
miglior_scheda = algoritmo_genetico(gym_dataset2, livello_di_abilita, luogo_di_allenamento)
print("Migliore Scheda di Allenamento Generata:")
for giorno, allenamento in miglior_scheda.items():
    print(f"\n{giorno}:")
    for esercizio in allenamento:
        print(esercizio['Title'])
#input di feedback dell'utente che può dire come gli sembra la scheda scegliendo fra le opzioni Ottima Troppo intensa Troppo facile cercando di migliorare la scelta del GA
feedback = input("\nFornisci il tuo feedback sulla scheda di allenamento (es. 'Ottima', 'Troppo intensa', 'Troppo facile'): ")
#creamo le schede contando il feedback
miglior_scheda_con_feedback = algoritmo_genetico(gym_dataset2, livello_di_abilita, luogo_di_allenamento, feedback)

print("Migliore Scheda di Allenamento Generata con Feedback:")
for giorno, allenamento in miglior_scheda_con_feedback.items():
    print(f"\n{giorno}:")
    for esercizio in allenamento:
        print(esercizio['Title'])
punteggio_scheda_con_feedback = fitness(miglior_scheda_con_feedback, livello_di_abilita, luogo_di_allenamento, pesi_iniziali, feedback)
print(f"Punteggio della miglior scheda di allenamento con feedback: {punteggio_scheda_con_feedback}")
